[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Trusted-AI/AIF360/blob/main/examples/sklearn/demo_learning_fair_representations.ipynb)

# Learning Fair Representations

The Learned Fair Representations method (LFR) aims to learn an encoding of the data which preserves as much information as possible while removing protected information. It does this by forming a probabilistic mapping of the data to a set of "prototypes" (clusters). The learned representation can be used by a downstream estimator (pre-processing) or as a linear model itself, using weights learned in the encoding process.

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import seaborn as sns
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from aif360.sklearn.metrics import * 
from aif360.sklearn.preprocessing import LearnedFairRepresentations
from aif360.sklearn.datasets.utils import standardize_dataset

from sklearn.metrics import *

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\torch\_functorch\deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')
pip install 'aif360[inFairness]'
pip install 'aif360[FairAdapt]'


In [3]:
RANDOM_STATE = 42

In [8]:
def fetch_dataset(protected_attributes, subset='all', *, data_home=None, cache=True, binary_race=True,
                usecols=None, dropcols=None, numeric_only=False, dropna=False):
    if subset not in {'train', 'test', 'all'}:
        raise ValueError("subset must be either 'train', 'test', or 'all'; "
                         "cannot be {}".format(subset))
    df = pd.read_csv('./../data/synth_data_for_training.csv')
    


    return standardize_dataset(df, prot_attr=protected_attributes,
                               target='checked',
                               usecols=usecols, dropcols=dropcols,
                               numeric_only=numeric_only, dropna=dropna)

Fetch Adult dataset, encode categorical features, and split into train/test.

In [9]:
protected_attributes = ["persoon_geslacht_vrouw", "typering_ind"]

X, y= fetch_dataset(protected_attributes=protected_attributes)


# X.index = y.index = pd.MultiIndex.from_arrays(X.index.codes, names=X.index.names)
y = pd.Series(y.factorize(sort=True)[0], index=y.index, name=y.name)
X = pd.get_dummies(X, prefix_sep='__', drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)
X_train

adres_aantal_brp_adres  \
persoon_geslacht_vrouw typering_ind                           
0                      0                                  6   
                       1                                  4   
1                      0                                  4   
0                      1                                  3   
                       1                                  3   
...                                                     ...   
                       1                                  6   
1                      1                                  6   
0                      0                                  2   
                       1                                  3   
                       1                                  3   

                                     adres_aantal_verschillende_wijken  \
persoon_geslacht_vrouw typering_ind                                      
0                      0                                             3   
                       1                                             2   
1                      0                                             2   
0                      1                                             2   
                       1                                             3   
...                                                                ...   
                       1                                             2   
1                      1                                             4   
0                      0                                             2   
                       1                                             2   
                       1                                             3   

                                     adres_aantal_verzendadres  \
persoon_geslacht_vrouw typering_ind                              
0                      0                                     1   
                       1                                     1   
1                      0                                     0   
0                      1                                     0   
                       1                                     0   
...                                                        ...   
                       1                                     0   
1                      1                                     1   
0                      0                                     1   
                       1                                     0   
                       1                                     0   

                                     adres_aantal_woonadres_handmatig  \
persoon_geslacht_vrouw typering_ind                                     
0                      0                                            0   
                       1                                            0   
1                      0                                            1   
0                      1                                            1   
                       1                                            2   
...                                                               ...   
                       1                                            1   
1                      1                                            1   
0                      0                                            1   
                       1                                            0   
                       1                                            1   

                                     adres_dagen_op_adres  \
persoon_geslacht_vrouw typering_ind                         
0                      0                             1012   
                       1                             5268   
1                      0                             1820   
0                      1                             9056   
                       1                             5246   
...               

Create some scoring functions we can use in a grid search. We will use the balanced accuracy version of the delta function to avoid getting stuck in the case of predicting all zeros.

In [10]:
from sklearn import metrics as skm

def discrimination(y_true, y_pred):
    return abs(statistical_parity_difference(y_true, y_pred, prot_attr=protected_attributes))

def delta(y_true, y_pred, use_bal_acc=False):
    if use_bal_acc:
        return balanced_accuracy_score(y_true, y_pred) - discrimination(y_true, y_pred)
    else:
        return accuracy_score(y_true, y_pred) - discrimination(y_true, y_pred)

min_disc = make_scorer(statistical_parity_difference, prot_attr=protected_attributes)
max_delta = skm.make_scorer(delta, use_bal_acc=True)

Use a grid search to find the best model. Note: this takes a while to run.

In [11]:

protected_attributes = ["persoon_geslacht_vrouw", "typering_ind"]


lfr = LearnedFairRepresentations(protected_attributes, n_prototypes=25, max_iter=10, random_state=RANDOM_STATE)
lfr.fit(X_train, y_train)


params = {'reconstruct_weight': [1e-2, 1e-3, 1e-4],
        'target_weight': [100, 1000],
        'fairness_weight': [0, 100, 1000]}
grid = GridSearchCV(lfr, params, scoring=max_delta, cv=3, n_jobs=-1).fit(X_train, y_train)
res = pd.DataFrame(grid.cv_results_)
res

AttributeError: 'str' object has no attribute 'decode'

Compare predictions from naive logistic regression, LFR (in-processing), and LFR (pre-processing) with a logistic regression classifier.

In [ ]:
y_pred_lfr = grid.predict(X_test)
gbc = GradientBoostingClassifier(random_state=RANDOM_STATE)
y_pred_lr = gbc.fit(X_train, y_train).predict(X_test)
y_pred_lfr_lr = gbc.fit(grid.transform(X_train), y_train).predict(X_test)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred_lr)
print(conf_matrix)

class_report = classification_report(y_test, y_pred_lr)
print(class_report)

[[2815   22]
 [ 175  150]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      2837
           1       0.87      0.46      0.60       325

    accuracy                           0.94      3162
   macro avg       0.91      0.73      0.78      3162
weighted avg       0.93      0.94      0.93      3162



In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred_lfr)
print(conf_matrix)

class_report = classification_report(y_test, y_pred_lfr)
print(class_report)

[[2837    0]
 [ 325    0]]
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      2837
           1       0.00      0.00      0.00       325

    accuracy                           0.90      3162
   macro avg       0.45      0.50      0.47      3162
weighted avg       0.80      0.90      0.85      3162



d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

Feel free to try this out with other datasets as well!